# Web Scraping scryfall.com

In [191]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# A Better Way

So, I tried a certian approach when I first started, which you can see at the end. I'm leaving it in for academic purposes. But, I've since discovered a better way, which you can see here.

The sets page has some good stuff! Like number of cards and release date! I need to get that!!!

Let's inspect the page in Chrome to see which tag has the info I want.

It looks like each row is contained within a \<tr> tag. So let's dissect one of these.

In [171]:
def clean_text(text):
    '''A regex to clean up text, by removing formatting characters and leading or trailing whitespace'''

#     text = re.sub(r'([\n])|[\r]|(^\s+)|(\s+$)', '', text) # Meant to be a combined regex, not working
    text = re.sub(r'[\n]', '', text) # Delete line breaks
    text = re.sub(r'[\r]', '', text) # Delete carriage returns with a space
    text = re.sub(r'(^\s+)|(\s+$)', '', text) # Delete all leading or trailing whitespace
    return text

# Ya, I could probably combine all these into one regex, and compile it. But I'm in a hurry

In [172]:
r = requests.get(r'https://www.scryfall.com/sets')

sets_data = r.text

sets_soup = BeautifulSoup(sets_data)

In [173]:
# I read that its MUCH more efficient to put all the data into a list or dict, then append to df or create a df

lst = []

for row in sets_soup.find_all('tr')[1:]:
    boxes = row.find_all('td')
    info = boxes[0].text.rsplit(' ', 1) + [boxes[1].text, boxes[2].text, boxes[1].find('a').get('href')]
    info = [clean_text(text) for text in info]
    lst.append(info)

lst

[['Unsanctioned', 'UND', '96', '2020-02-29', 'https://scryfall.com/sets/und'],
 ['Unsanctioned Tokens',
  'TUND',
  '6',
  '2020-02-29',
  'https://scryfall.com/sets/tund'],
 ['Theros Beyond Death',
  'THB',
  '358',
  '2020-01-24',
  'https://scryfall.com/sets/thb'],
 ['Theros Beyond Death Promos',
  'PTHB',
  '136',
  '2020-01-24',
  'https://scryfall.com/sets/pthb'],
 ['Theros Beyond Death Tokens',
  'TTHB',
  '14',
  '2020-01-24',
  'https://scryfall.com/sets/tthb'],
 ['Judge Gift Cards 2020',
  'J20',
  '6',
  '2020-01-01',
  'https://scryfall.com/sets/j20'],
 ['MagicFest 2020',
  'PF20',
  '6',
  '2020-01-01',
  'https://scryfall.com/sets/pf20'],
 ['Secret Lair Drop',
  'SLD',
  '60',
  '2019-12-02',
  'https://scryfall.com/sets/sld'],
 ['Secret Lair Drop Promos',
  'PSLD',
  '34',
  '2019-12-02',
  'https://scryfall.com/sets/psld'],
 ['Historic Anthology 1',
  'HA1',
  '20',
  '2019-11-21',
  'https://scryfall.com/sets/ha1'],
 ['Game Night 2019',
  'GN2',
  '64',
  '2019-11-15',

In [174]:
sets = pd.DataFrame(lst, columns = ['name', 'abbreviation', 'num_cards', 'release_date', 'url'])
sets.num_cards = sets.num_cards.astype('int64')
sets.release_date = pd.to_datetime(sets.release_date)
sets

,name,abbreviation,num_cards,release_date,url
0,Unsanctioned,UND,96,2020-02-29,https://scryfall.com/sets/und
1,Unsanctioned Tokens,TUND,6,2020-02-29,https://scryfall.com/sets/tund
2,Theros Beyond Death,THB,358,2020-01-24,https://scryfall.com/sets/thb
3,Theros Beyond Death Promos,PTHB,136,2020-01-24,https://scryfall.com/sets/pthb
4,Theros Beyond Death Tokens,TTHB,14,2020-01-24,https://scryfall.com/sets/tthb
...,...,...,...,...,...
575,Intl. Collectors’ Edition,CEI,302,1993-12-10,https://scryfall.com/sets/cei
576,Collectors’ Edition,CED,302,1993-12-10,https://scryfall.com/sets/ced
577,Unlimited Edition,2ED,302,1993-12-01,https://scryfall.com/sets/2ed
578,Limited Edition Beta,LEB,302,1993-10-04,https://scryfall.com/sets/leb


There's a few entries that don't have the full urls, so let's fix that.

In [175]:
mask = sets.url.apply(lambda url: url[:5] != 'https')
sets.url[mask] = sets.url[mask].apply(lambda url: 'https://scryfall.com' + url)

C:\Users\evant\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


I could add info about which are "subsets", like promos and tokens, as those tags have an "indent class". But I won't worry about that now.

Once I learn SQL, that might be a better way to keep track of this stuff

# UND

Ok, let's try to grab all the info we can for one set. Then we'll generalize that technique for all sets. I guess we'll store each in a seperate data frame for now. Though I really am starting to think I need to use SQL.

Each set page has pictures of every card (by default). I can activate a checklist view by adding '?as=checklist', which makes it easier to get the data I want.

We end up with a similar html structure to the sets page, so I'll reuse most of that code.

In [176]:
r = requests.get(r'https://scryfall.com/sets/und?as=checklist')

data = r.text

soup = BeautifulSoup(data)

In [177]:
lst = []

for row in soup.find_all('tr')[1:]:
    boxes = row.find_all('td')
    info = [boxes[i].text for i in range(11)]
    info = [clean_text(text) for text in info]
    lst.append(info)

lst

[['UND',
  '1',
  'Adorable Kitten',
  '{W}',
  'Host Creature',
  'C',
  'EN',
  'Andrea Radeck',
  '$0.14',
  '€0.06',
  ''],
 ['UND',
  '2',
  'AWOL',
  '{2}{W}',
  'Instant',
  'C',
  'EN',
  'Stephen Tappin',
  '$0.13',
  '€0.04',
  ''],
 ['UND',
  '3',
  'Emcee',
  '{2}{W}',
  'Creature',
  'U',
  'EN',
  'Quinton Hoover',
  '$0.17',
  '€0.04',
  ''],
 ['UND',
  '4',
  'Flavor Judge',
  '{1}{W}',
  'Creature',
  'R',
  'EN',
  'Mike Burns',
  '$0.25',
  '€0.19',
  ''],
 ['UND',
  '5',
  'Frankie Peanuts',
  '{2}{W}{W}',
  'Lgd. Creature',
  'R',
  'EN',
  'Thomas M. Baxa',
  '$0.58',
  '€0.16',
  ''],
 ['UND',
  '6',
  'GO TO JAIL',
  '{W}',
  'Enchantment',
  'C',
  'EN',
  'Marco Teixeira',
  '$0.13',
  '€0.04',
  ''],
 ['UND',
  '7',
  'Humming-',
  '',
  'Creature',
  'C',
  'EN',
  'Mark Behm',
  '$0.14',
  '€0.05',
  ''],
 ['UND',
  '8',
  'Knight of the Hokey Pokey',
  '{W}{W}',
  'Creature',
  'C',
  'EN',
  'Kev Walker',
  '$0.13',
  '€0.02',
  ''],
 ['UND',
  '9',
  "Lo

In [178]:
cards = pd.DataFrame(lst, columns = ['set', 'collectors_num', 'name', 'cost', 'type', 'rarity', 'language', 'artist', 'usd', 'eur', 'tix'])
cards

,set,collectors_num,name,cost,type,rarity,language,artist,usd,eur,tix
0,UND,1,Adorable Kitten,{W},Host Creature,C,EN,Andrea Radeck,$0.14,€0.06,
1,UND,2,AWOL,{2}{W},Instant,C,EN,Stephen Tappin,$0.13,€0.04,
2,UND,3,Emcee,{2}{W},Creature,U,EN,Quinton Hoover,$0.17,€0.04,
3,UND,4,Flavor Judge,{1}{W},Creature,R,EN,Mike Burns,$0.25,€0.19,
4,UND,5,Frankie Peanuts,{2}{W}{W},Lgd. Creature,R,EN,Thomas M. Baxa,$0.58,€0.16,
...,...,...,...,...,...,...,...,...,...,...,...
91,UND,92,Swamp,,Basic Land,C,EN,Adam Paquette,$0.27,€1.99,
92,UND,93,Mountain,,Basic Land,C,EN,Rob Alexander,$0.19,€0.03,
93,UND,94,Mountain,,Basic Land,C,EN,Adam Paquette,$0.18,€1.85,
94,UND,95,Forest,,Basic Land,C,EN,Mark Poole,$0.29,€0.02,


We did it!! Now, time to get ALL THE CARDS!!!

# ALL THE CARDS!

In [198]:
start_time = time.time()

lst = []
for index, site in sets.url.iteritems():
    r = requests.get(f'{site}/?as=checklist')
    data = r.text
    soup = BeautifulSoup(data)
    
    for row in soup.find_all('tr')[1:]:
        boxes = row.find_all('td')
        info = [boxes[i].text for i in range(11)]
        info = [clean_text(text) for text in info]
        lst.append(info)
        
cards = pd.DataFrame(lst, columns = ['set', 'collectors_num', 'name', 'cost', 'type', 'rarity', 'language', 'artist', 'usd', 'eur', 'tix'])   
run_time = time.time() - start_time
print(f'--- This program took {run_time} seconds to run. ---')

--- This program took 462.6313302516937 seconds to run. ---


In [199]:
cards

,set,collectors_num,name,cost,type,rarity,language,artist,usd,eur,tix
0,UND,1,Adorable Kitten,{W},Host Creature,C,EN,Andrea Radeck,$0.14,€0.06,
1,UND,2,AWOL,{2}{W},Instant,C,EN,Stephen Tappin,$0.13,€0.04,
2,UND,3,Emcee,{2}{W},Creature,U,EN,Quinton Hoover,$0.17,€0.04,
3,UND,4,Flavor Judge,{1}{W},Creature,R,EN,Mike Burns,$0.25,€0.19,
4,UND,5,Frankie Peanuts,{2}{W}{W},Lgd. Creature,R,EN,Thomas M. Baxa,$0.58,€0.16,
...,...,...,...,...,...,...,...,...,...,...,...
55253,LEA,291,Swamp,,Basic Land,C,EN,Dan Frazier,$19.41,€23.52,
55254,LEA,292,Mountain,,Basic Land,C,EN,Douglas Shuler,$29.23,€15.00,
55255,LEA,293,Mountain,,Basic Land,C,EN,Douglas Shuler,$19.98,€15.00,
55256,LEA,294,Forest,,Basic Land,C,EN,Christopher Rush,$30.73,€14.97,


In [200]:
cards.to_csv('D:/code/Data/scryfall/cards.csv', header = True)

In [209]:
cards.groupby('artist').count().name.sort_values()

artist
Sarah Keortge            1
Mark Purvis              1
Darbury Stenderu         1
Mark Price               1
Wesley Burt & コーヘー       1
                      ... 
Greg Staples           627
Dan Frazier            677
Mark Tedin             685
Kev Walker            1027
John Avon             1103
Name: name, Length: 916, dtype: int64

In [221]:
cards.dtypes

set               object
collectors_num    object
name              object
cost              object
type              object
rarity            object
language          object
artist            object
usd               object
eur               object
tix               object
dtype: object

In [220]:
def to_float(x):
    try:
        return float(x[1:])
    except:
        return x

cards.usd.apply(to_float).plot.hist()

TypeError: no numeric data to plot

# My original, not that great, approach

In [ ]:
url = r'https://www.scryfall.com'

r = requests.get(url)

data = r.text

soup = BeautifulSoup(data)

In [ ]:
print(soup.prettify()[:1000])

In [ ]:
links = soup.find_all('a')

display(f'There are {len(links)} links')

for link in links:
    print(link.get('href'))

Ok, that's a lot of pages. The "sets" page looks promising. I think that will have every card. Let's go there!

In [ ]:
r = requests.get(f'{url}/sets')

sets_data = r.text

sets_soup = BeautifulSoup(sets_data)

In [ ]:
# print(sets_soup.prettify())

In [ ]:
sets_links = sets_soup.find_all('a')

display(f'There are {len(sets_links)} links')

for link in sets_links[:100]:
    print(link.get('href'))

Hmm, a lot of repeats and a lot of different formats. Let's fix all the links to be the full link, then create a list without duplicates.

In [ ]:
def fix_link(link):
    try:
        if link[:5] == 'https':
            return link
        elif link[0] == '/':
            return f'{url}{link}'
    except:
        print('Empty Link')

In [ ]:
sets_list = [link.get('href') for link in sets_links]

fixed_sets_list = [fix_link(string) for string in sets_list]
fixed_sets_list = list(set(fixed_sets_list))
fixed_sets_list

Looking good. A few more things to work on:
1. Some have www and some don't. Let's add that to all of them.
2. There will be more duplicates after we do that. (I suspect that we could just drop all that don't have www, but it doesn't hurt to be careful.
3. There are a few bad ones, like reddit and none. Let's lose those as well.

In [ ]:
fixed_sets_list.remove(None)

In [ ]:
fixed_sets_list

In [ ]:
# Join them all into one string
# Seems easer to do regex when it's all one string. 
# Should investigate this more. Am I just bad, or is this a good way to do it?

csv = ','.join(fixed_sets_list)

csv = re.sub(r'/scryfall', r'/www.scryfall', csv)

fixed_sets_list = csv.split(',')

In [ ]:
fixed_sets_list = list(set(fixed_sets_list))
fixed_sets_list

Ok, almost there. There's still a few unwanted sites in there. Let's get rid of every site which doesn't have a "www.scryfall.com/sets" in it

In [ ]:
csv = ','.join(fixed_sets_list)

regex = r'[^,]*www.scryfall.com/sets[^,]*'

csv = re.findall(regex, csv)

fixed_sets_list = csv
fixed_sets_list

Now, I know this isn't yet correct, because on that page it says that there's 579 sets. So what's gone wrong?

Well, some of the set pages have additional "/" after the set designation. So it's some kind of subpage. 

Oh, I see, it's for each language. Well, I believe the default for each is English, so let's get rid of everything after the set description.

In [ ]:
csv = ','.join(fixed_sets_list)

regex = r'(/sets/[^/,]*?)/[^/,]*?,'

# re.findall(regex, csv, flags = re.VERBOSE)

csv = re.sub(regex, r'\1,', csv, flags = re.VERBOSE)

final_sets_list = csv.split(',')

final_sets_list

YES! Looking good! Now I just need to remove the duplicates again (could have waited until now, but I just like cleaning things).

In [ ]:
final_sets_list = list(set(final_sets_list))
final_sets_list

So close!!! We ended up with 574 links. But there are actually 579 sets (According to the bottom of the page). We'll I'm not sure what I missed, but I think there's a better way to handle this stuff.